### Задание № 1 

##### Эссе с элементами кодинга:
### «Что такое категориальный признак, как с ним бороться и зачем».
##### К каждому пункту, описывающему определенный метод кодировки, писать код, как бы вы это делали на условном признаке X.

   Категориальный (факторный, номинальный) признак - это признак, который описывает свойство объекта выборки. 
Например: "цвет" - "зеленый", "красный"; "форма" - "квадратный", "круглый"; и т.п..
То есть существует общее признаковое описание - "цвет" (или "форма"), которое содержит 
в себе категории признаков - "зеленый", "красный", и т.д. 
Отсюда и название - "Категориальные признаки".
   Особенность категориальных признаков в том, что с ними нельзя проводить математические и
логические операции типа сложение, сравнение на "больше"-"меньше", получение среднего 
зачения и т.п.. Их можно только сравнивать на тождественность. То есть проводить операции типа "признак объекта 1 равен (не равен) признаку объекта 2".


   Из вышесказанного очевидно, что оперирование категориальными признаками в исходном виде 
для решения задач линейной регрессии невозможно. Необходима перекодировка. 
    Создадим для демонстрации выборку, в которой признак "вес" - вещественный признак, 
"год выпуска" - категориальный, но это число, "цвет" - категориальный.

In [80]:
import pandas as pd
data = pd.DataFrame({'цвет': ['красный', 'зеленый', 'красный', 'синий'],
                    'год выпуска': [2017, 2017, 2018, 2017],
                    'вес': [10.2, 11.7, 14.1, 9.0]})
data.head()

,вес,год выпуска,цвет
0,10.2,2017,красный
1,11.7,2017,зеленый
2,14.1,2018,красный
3,9.0,2017,синий


##### Как первично автоматически отсортировать категориальные признаки и по каким маркерам  

In [81]:
# для первичной автоматической сортировки признаков и выделения из них категориальных можно 
# отобрать те признаки, значения которых мало уникальны (значение уникальности выбираем 
# произвольно) и/или имеют строковый тип данных
def find_categorical_features(data):
    for name in data.columns:
        feature = 'признак - '
        feature += name
        if (type(data[name][0]) == str):
            feature += ', тип данных - строковый'
        if (data[name].nunique() <= 2):
            feature += ', мало уникальных значений'
        if feature != name:
            print feature
find_categorical_features(data)

признак - вес
признак - год выпуска, мало уникальных значений
признак - цвет, тип данных - строковый


итак, мы видим, что у признака "год выпуска" мало уникальных значений, 
несмотря на то, что тип данных - целое число. а у признака "цвет" - строковый тип данных. малое число уникальных значений, как и строковый тип данных - индикаторы категориалього признака. 

##### Кодировка заменой признака на число  

Простейший способ кодировки - замена категорий на целые числа. для этого используется 
кодировщик sklearn.preprocessing.LabelEncoder
Продемонстрируем его работу на колонке "цвет":

In [82]:
from sklearn.preprocessing import LabelEncoder
to_integr_encoder = LabelEncoder()
to_integr_encoder.fit(data['цвет'])
data['цвет кодированный'] = to_integr_encoder.transform(data['цвет'])
data.head()

,вес,год выпуска,цвет,цвет кодированный
0,10.2,2017,красный,1
1,11.7,2017,зеленый,0
2,14.1,2018,красный,1
3,9.0,2017,синий,2


кодирование произошло, но этот способ сильно упрощает данные, что ведет к искажению информации.

##### Наивное кодирование 

Универсальным способом перекодировки столбца с признаковым описанием без потери информации 
считается замена этого столбца на бинарную матрицу. Этот способ еще
называют наивным (dummy) или  one-hot encoding. Для его реализации используют sklearn.preprocessing.OneHotEncoder 

In [83]:
from sklearn.preprocessing import OneHotEncoder 
to_binary_matrix_encoder = OneHotEncoder(sparse=False)
new_features = to_binary_matrix_encoder.fit_transform(data['цвет кодированный'].
                                                     values.reshape(-1, 1))
tmp = pd.DataFrame(new_features, columns = 
                   ['цвет кодированный='+str(i) for i in range(new_features.shape[1]) ])
print new_features
data_new = pd.concat([data, tmp], axis=1)
data_new.head()

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


,вес,год выпуска,цвет,цвет кодированный,цвет кодированный=0,цвет кодированный=1,цвет кодированный=2
0,10.2,2017,красный,1,0.0,1.0,0.0
1,11.7,2017,зеленый,0,1.0,0.0,0.0
2,14.1,2018,красный,1,0.0,1.0,0.0
3,9.0,2017,синий,2,0.0,0.0,1.0


кодирование произошло, но этот способ увеличивает объем данных.

##### "Умное" кодирование

чтобы избежать увеличения объема данных, можно кодировать через интерпретацию 
связанных данных. например, мы можем закодировать "цвет"-"красный" через средний вес объектов "красного" цвета в выборке. 

In [86]:
def code_by_mean_weight(data, cat_feature, real_feature):
    return (data[cat_feature].map(data.groupby(cat_feature)[real_feature].mean()))

data['цвет кодированный через средний вес'] = code_by_mean_weight(data, 'цвет', 'вес')
data.head()

,вес,год выпуска,цвет,цвет кодированный через средний вес
0,10.2,2017,красный,12.15
1,11.7,2017,зеленый,11.70
2,14.1,2018,красный,12.15
3,9.0,2017,синий,9.00


In [115]:
# и еще один способ наивного преобразования - при помощи метода DictVectorizer
import pandas as pd
from sklearn.feature_extraction import DictVectorizer as DV

data = pd.DataFrame({'цвет': ['красный', 'зеленый', 'красный', 'синий'],
                    'вес': [10.2, 11.7, 14.1, 9.0]})

encoder = DV(sparse = False)
categorical_columns = ['цвет']
categorical_data = data_formated[categorical_columns]
categorical_data_vectorized = encoder.fit_transform(categorical_data.T.to_dict().values())
print 'initial categorial features column'
print data['цвет']
print 'vectorized data'
print categorical_data_vectorized

initial categorial features column
0    красный
1    зеленый
2    красный
3      синий
Name: цвет, dtype: object
vectorized data
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


# Задание № 2 

##### Эссе в 1 предложение: 
### «Что такое задача регрессии».

Задача регрессии - создание математической модели, которая выстраивает связь между группой факторов, полученных при наблюдении за ними разными способами (как на одном (множестве) объектов, так и в один (разные) период времени), и неким объясняемым событием или объясняемой величиной, связанными с этими факторами общим объектом.